# Script to understand and extract Marconi100 gpu power metrics

I'm using the jobs from `filter123_all_jobs_aggmetrics.csv.gz` (output from `marconi_jobs_extract_power_metrics.py`)

In [1]:
import pandas as pd
import numpy as np

## Time constants
NANOSECONDS_ONE_SECOND = 1e9

NB_GPUS = 4

#df_jobs = pd.read_csv("/home/dancarastan/Dropbox/notebook/posdoc-grenoble/energy-analysis-retake/marconi100-data/filter123_all_jobs_aggmetrics.csv.gz")

df_jobs = pd.read_csv("/home/dcarastandossantos/datamove_storage/danilo-carastan-santos/exadata/data/year_month=22-05/plugin=job_table/metric=job_info_marconi100/a_0_filter123_aggmetrics.csv")

power_filepaths = ["/home/dcarastandossantos/datamove_storage/danilo-carastan-santos/exadata/data/year_month=22-05/plugin=ganglia_pub/metric=Gpu"+str(x)+"_power_usage/a_0.parquet" for x in range(NB_GPUS)]

#power_filepaths = ["../Marconi-test/plugin=ganglia_pub/metric=Gpu"+str(x)+"_power_usage/a_0.parquet" for x in range(NB_GPUS)]
#print(power_filepaths)

df_power = pd.concat([pd.read_parquet(x).assign(gpu=y) for x, y in zip(power_filepaths, range(NB_GPUS))]).reset_index(drop=True)

df_power["timestamp_seconds"] = pd.to_datetime(df_power['timestamp']).astype(int) / NANOSECONDS_ONE_SECOND
df_jobs["submit_time"] = pd.to_datetime(df_jobs['submit_time']).astype(int) / NANOSECONDS_ONE_SECOND
df_jobs["start_time"] = pd.to_datetime(df_jobs['start_time']).astype(int) / NANOSECONDS_ONE_SECOND
df_jobs["end_time"] = pd.to_datetime(df_jobs['end_time']).astype(int) / NANOSECONDS_ONE_SECOND

df_power['node'] = pd.to_numeric(df_power['node'])
df_power['value'] = pd.to_numeric(df_power['value'])

#df_power = pd.concat([pd.read_parquet(x).insert(0, "gpu", y) for x,y in zip(power_filepaths, range(NB_GPUS))])

df_power

/tmp/ipykernel_26736/2911483124.py:11: DtypeWarning: Columns (63,90) have mixed types. Specify dtype option on import or set low_memory=False.
  df_jobs = pd.read_csv("/home/dcarastandossantos/datamove_storage/danilo-carastan-santos/exadata/data/year_month=22-05/plugin=job_table/metric=job_info_marconi100/a_0_filter123_aggmetrics.csv")


,timestamp,value,node,gpu,timestamp_seconds
0,2022-05-07 10:21:31+00:00,51.374001,72,0,1.651919e+09
1,2022-05-07 10:21:52+00:00,51.374001,72,0,1.651919e+09
2,2022-05-07 10:22:13+00:00,51.374001,72,0,1.651919e+09
3,2022-05-07 10:22:34+00:00,51.374001,72,0,1.651919e+09
4,2022-05-07 10:22:55+00:00,51.374001,72,0,1.651919e+09
...,...,...,...,...,...
452109085,2022-05-05 16:40:37+00:00,280.036011,700,3,1.651769e+09
452109086,2022-05-05 16:40:58+00:00,195.654007,700,3,1.651769e+09
452109087,2022-05-05 16:41:20+00:00,195.654007,700,3,1.651769e+09
452109088,2022-05-05 16:41:29+00:00,198.216995,700,3,1.651769e+09


In [32]:
import numpy as np
import ast

group = df_power.groupby(by="node")

#for debugging
#result = [group.get_group(x[0])["timestamp"].between(x[1], x[2]).value_counts() for x in sample[["node", "start_time", "end_time"]].values.tolist()]

#i know that x[0] is the node list, x[1] is start_time, and x[2] is end_time
#each element of the list is a subseries of the total_power of a certain mode
#with "1" at the timestamps where the "job_id" ran
# i replace the "0"s (where the timestamp is not betweenx[1], x[2]) with nans just to remove these values with dropna.

# for debug
#sample = df_jobs[0:1000].copy().reset_index(drop=True)

# the real run
sample = df_jobs
sample["node"] = [ast.literal_eval(x) for x in sample['nodes']]

#print(catch(group.get_group,179))
#print (sample['nodes'])
#print (sample["node"])

group_keys = np.asarray([key for key, _ in group])

# list comprehension debug
#for x in sample["node"].values.tolist():
#    for y in x:
#        print(np.isin(y,group_keys))


sample["has_profile"] = [all([np.isin(y,group_keys) for y in x]) for x in sample["node"].values.tolist()]

#print(sample["has_profile"])

#print(group.groups.keys())
#print(sample["has_profile"])

print("Number of jobs with no GPU profile: ",len(sample[sample["has_profile"]==False]))

sample = sample[sample["has_profile"]==True]

# Get the power profile in the intersection between start_time and end_time
result = [[[group.get_group(y)["timestamp_seconds"].between(x[1], x[2]).astype(int).replace(0, np.nan).dropna().astype(str) for y in x[0]], x[3]] for x in sample[["node", "start_time", "end_time", "job_id"]].values.tolist()]

#print(result)

# now i replace the "1"s with the "job_id"
#each element of the list is a subseries of the df_power of a certain node
#with the "job_id" value at the timestamps where the "job_id" ran
#result2 = [pd.concat([y.replace("1.0", str(x[1])) for y in x[0]]) for x in result]


result2 = [df_power.iloc[pd.concat([y for y in x[0]]).index.values].assign(job_id=x[1]) for x in result]


#result2[1]

final_result = pd.concat(result2)
final_result

#finally i concatenate each of the series by index
# while joining the values where indices overlap
# i hope the indices here are consistent with the indices in df_total_power 
#concatenated_series = pd.concat(result2).groupby(level=0).apply(','.join).replace(to_replace=r'^(0,)*0$', value="0", regex=True)
            
#concatenated_series = pd.concat(result2).groupby(level=0).apply(','.join).rename("job_ids")
#print(concatenated_series)

# use the below code for the full run
#final_result = df_power.merge(concatenated_series, left_index=True, right_index=True, how="left")

#use the below code for a test run
#final_result = df_power.iloc[result2[0].index.values]
#final_result = df_power.merge(concatenated_series, left_on="timestamp_seconds", right_index=True, how="inner")


Number of jobs with no GPU profile:  13344


,timestamp,value,node,gpu,timestamp_seconds,job_id
80310914,2022-05-28 03:31:57+00:00,132.369995,291,0,1.653709e+09,3915009
80310915,2022-05-28 03:32:09+00:00,132.369995,291,0,1.653709e+09,3915009
80310916,2022-05-28 03:32:31+00:00,150.716003,291,0,1.653709e+09,3915009
80310917,2022-05-28 03:32:54+00:00,153.740997,291,0,1.653709e+09,3915009
80310918,2022-05-28 03:33:17+00:00,142.917999,291,0,1.653709e+09,3915009
...,...,...,...,...,...,...
421380071,2022-05-20 02:22:17+00:00,152.056000,878,3,1.653013e+09,4997220
421380072,2022-05-20 02:22:39+00:00,146.112000,878,3,1.653013e+09,4997220
421380073,2022-05-20 02:22:51+00:00,146.112000,878,3,1.653013e+09,4997220
421380074,2022-05-20 02:23:12+00:00,154.005005,878,3,1.653013e+09,4997220


In [34]:
sample.to_csv("/home/dcarastandossantos/datamove_storage/danilo-carastan-santos/exadata/data/year_month=22-05/plugin=job_table/metric=job_info_marconi100/a_0_filter123_aggmetrics_gpu_jobs.csv.gz",index=False)
final_result.to_csv("/home/dcarastandossantos/datamove_storage/danilo-carastan-santos/exadata/data/year_month=22-05/plugin=ganglia_pub/a_0_filter123_aggmetrics_gpu_power_usage.csv.gz", index=False)
